# Data Cleaning with PySpark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, regexp_replace, regexp_extract, monotonically_increasing_id


spark = SparkSession.builder.appName("Data_cleaning").getOrCreate()

## 1. Data Reading and Loading

In [ ]:
df = spark.read.csv("population.csv", header=True, sep=";")

In [ ]:
df.take(3)

In [ ]:
df.show(3)

In [ ]:
df.collect()

## 2. Data Description

In [ ]:
df.printSchema()

In [ ]:
df.columns

In [ ]:
df.schema.names

In [ ]:
df.describe()

In [ ]:
df.describe().show()

In [ ]:
df.summary().show()

In [ ]:
df.count()

In [ ]:
len(df.columns)

## 3. Handling Missing Data

In [ ]:
df.filter(col("with_migration_background").isNotNull()).count()

In [ ]:
df.filter(col("with_migration_background").isNull()).count()

In [ ]:
df.columns

In [ ]:
for i in df.columns:
    print(f"{i} : {df.filter(col(i).isNull()).count()}")

In [ ]:
df = df.fillna("deleting")
df.show(3)

In [ ]:
df.count()

## 4. Data Filtering and Cleaning

In [ ]:
df.filter(col("total") == "deleting").show()

In [ ]:
df.drop(col("total") == "deleting")

In [ ]:
df.filter(col("total") == "deleting").count()

In [ ]:
df.filter(col("with_migration_background") == "deleting").count()

In [ ]:
df.count()

In [ ]:
df = df.where(df.total != "deleting")

In [ ]:
df.filter(col("total") == "deleting").show()

In [ ]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [ ]:
df = df.withColumn("index", monotonically_increasing_id())
df.show(3)

In [ ]:
column_list = df.columns[:20]

In [ ]:
df = df.select("index", *column_list)
df.show(3)

In [ ]:
df.filter((df.index==1) | (df.index==2)).show()

In [ ]:
df.show(3)

## 5. Data Manipulation

In [ ]:
df_postcode = df.filter(col("index") <= 4).withColumn("postcode", regexp_replace(col("postcode"), '[^a-zA-Z.]+', ""))
df_postcode.show()

In [ ]:
df_postcode.count()

In [ ]:
df_postcode_last = df.filter(col("index") > 4).withColumn("postcode", regexp_replace(col("postcode"), '[^0-9.]+', ""))
df_postcode_last.show(3)

In [ ]:
df_postcode_last.count()

In [ ]:
df = df_postcode.union(df_postcode_last)

In [ ]:
df.show(10)

In [ ]:
for col_name in df.columns:
    if col_name == "index":
        df = df.drop(col_name)
    elif col_name != "postcode":
        df = df.withColumn(col_name, regexp_replace(col(col_name), '[^0-9]+', ''))

df.show(3)

## 6. Data Type Conversions

In [ ]:
df = df.withColumn("year", col("year").cast("int"))

In [ ]:
for col_name in df.columns:
    if col_name != "year" and col_name != "postcode":
        df = df.withColumn(col_name, col(col_name).cast("int"))

df.printSchema()

In [ ]:
df.summary().show()

In [ ]:
df.show()

## 7. Data Merging

In [ ]:
df_nl = spark.read.csv("Nederland.csv", header=True)
df_nl.show(3)

In [ ]:
df_nl = df_nl.drop(col("Huisnummer"))
df_nl.show(3)

In [ ]:
df_nl = df_nl.withColumn("PC6", regexp_extract(df_nl["PC6"], r'^(.{4})', 1))
df_nl.show(3)

In [ ]:
df_nl = df_nl.dropDuplicates(['PC6'])
df_nl.show()

In [ ]:
df_nl.printSchema()

In [ ]:
df = df.join(df_nl, df.postcode == df_nl.PC6, "inner").drop("PC6")

In [ ]:
df.show(3)